<a href="https://colab.research.google.com/github/boujrafh/BackendConsumingRestApiToken/blob/master/Copie_de_evalutate_strat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install mplfinance
import mplfinance as mpf
!pip install talib-binary
import talib as ta
from datetime import datetime

In [ ]:
data = pd.read_json("/content/drive/MyDrive/dev/BNB-USDT_1m.json")
df = data.rename(columns={0: "date", 1: "open", 2: "high", 3: "low", 4: "close", 5: "volume"}, inplace=False)


In [ ]:
def add_flag(dataframe, add_plots, ref_name, price_ref_name, color, marker, to_add=True, shift_percent=0.01):
    if ref_name in dataframe:
        have_indicator = dataframe[ref_name].isin([1]).any().any()
        if have_indicator:
            signal_indexes = np.where(dataframe[ref_name] == 1)
            for ssi in signal_indexes[0]:
                if to_add:
                    dataframe[ref_name][ssi] = (
                            dataframe.iloc[ssi][price_ref_name] + dataframe.iloc[ssi][
                        price_ref_name] * shift_percent)
                else:
                    dataframe[ref_name][ssi] = (
                            dataframe.iloc[ssi][price_ref_name] - dataframe.iloc[ssi][
                        price_ref_name] * shift_percent)

            flags = mpf.make_addplot(dataframe[ref_name], panel=0, ylabel=ref_name, color=color,
                                      type='scatter',
                                      markersize=150, marker=marker)

            add_plots.append(flags)


In [ ]:
df_part = df.iloc[0:].copy()

In [ ]:
add_plots = []

In [ ]:
# Stoch
df_part["slowk"], df_part["slowd"] =  ta.STOCH(df_part.high, df_part.low, df_part.close)

slowk = mpf.make_addplot(df_part["slowk"], panel=1, ylabel='slowk', color='blue')
add_plots.append(slowk)
slowd = mpf.make_addplot(df_part["slowd"], panel=1, ylabel='slowd', color='red')
add_plots.append(slowd)

In [ ]:
# rsi14
df_part["rsi14"] =  ta.RSI(df_part.close, 14)

rsi14 = mpf.make_addplot(df_part["rsi14"], panel=2, ylabel='rsi14', color='blue')
add_plots.append(rsi14)

# rsi28
df_part["rsi28"] =  ta.RSI(df_part.close, 28)

rsi28 = mpf.make_addplot(df_part["rsi28"], panel=2, ylabel='rsi28', color='red')
add_plots.append(rsi28)

In [ ]:
# TEMA - Triple Exponential Moving Average
df_part['tema'] = ta.TEMA(df_part.close, 9)
tema = mpf.make_addplot(df_part["tema"], panel=0, ylabel='tema', color='c', markersize=150)
add_plots.append(tema)

In [ ]:
#ema10
df_part["ema10"] = ta.EMA(df_part.close, 10)
ema10 = mpf.make_addplot(df_part["ema10"], panel=0, ylabel='ema10', color='red', markersize=150)
add_plots.append(ema10)

#ema50
df_part["ema50"] = ta.EMA(df_part.close, 50)
ema50 = mpf.make_addplot(df_part["ema50"], panel=0, ylabel='ema50', color='orange', markersize=150)
add_plots.append(ema50)


In [ ]:
# Bollinger Bands
df_part["bb_upperband"], df_part["bb_middleband"], df_part["bb_lowerband"] =ta.BBANDS(df_part.close, 20, 2, 2)

bb_upperband = mpf.make_addplot(df_part["bb_upperband"], panel=0, ylabel='bb_upperband', color='red',
                                linestyle='dashdot')
add_plots.append(bb_upperband)

bb_middleband = mpf.make_addplot(df_part["bb_middleband"], panel=0, ylabel='bb_middleband',
                                  color='blue',
                                  linestyle='dashdot')
add_plots.append(bb_middleband)

bb_lowerband = mpf.make_addplot(df_part["bb_lowerband"], panel=0, ylabel='bb_lowerband',
                                color='darkgreen',
                                linestyle='dashdot')
add_plots.append(bb_lowerband)

In [ ]:
# df_part.loc[
# (
#   (df_part['ema10'] > df_part['ema50'])&
#   (df_part['tema'] >= df_part['ema50'] + df_part['ema50'] * 0.0005)&
#   (df_part['tema'] > df_part['ema10']) &
#   (df_part['low'] < df_part['ema50']) &
#   (df_part['slowk'] < 75) &
#   (df_part['slowk'] > df_part['slowd'])
# ),
# "open_indicator"] = 1
# print(df_part.head())


In [ ]:
df_part.loc[
(
  (df_part['tema'] >= df_part['ema50'] + df_part['ema50'] * 0.0005)&
  (df_part['slowk'] > 40)&
  (df_part['rsi14'] > 30)&
  (df_part['rsi14'].shift(1) < df_part['rsi14'])&
  # (df_part['ema10'].shift(1) < df_part['ema10'])&
  (df_part['tema'].shift(1) < df_part['tema'])&
  (df_part['low'] > df_part['bb_lowerband']) &
  (df_part['high'] < df_part['bb_middleband']) &
  (
    (df_part['low'].shift(1)  <= df_part['bb_lowerband'].shift(1)) | 
    (df_part['low'].shift(2)  <= df_part['bb_lowerband'].shift(2)) |
    (df_part['low'].shift(3)  <= df_part['bb_lowerband'].shift(3)) |
    (df_part['low'].shift(4)  <= df_part['bb_lowerband'].shift(4))
  )
),
"open_indicator"] = 1
print(df_part[df_part["open_indicator"] == 1].head())

               date     open  ...  bb_lowerband  open_indicator
678   1609499880000  38.7125  ...     38.651124             1.0
679   1609499940000  38.7259  ...     38.647069             1.0
680   1609500000000  38.7547  ...     38.645652             1.0
2371  1609601460000  38.0942  ...     38.068003             1.0
3756  1609684560000  41.0045  ...     40.872015             1.0

[5 rows x 17 columns]


In [ ]:
df_part["won"] =  np.nan
for i, cell in enumerate(df_part.iterrows()):
    if cell[1]["open_indicator"] > 0:
        open_price = cell[1]["open"]
        for c, r in df_part.iloc[i+1:i+120].iterrows():
            if r["open"] < open_price - open_price*0.005:
                break
            if r["open"] >= open_price + open_price*0.005:
                df_part.at[cell[0], "won"] = 1
                break
print(df_part[df_part["open_indicator"] == 1].head())
print(df_part[df_part["won"] == 1].head())


               date     open     high  ...  bb_lowerband  open_indicator  won
678   1609499880000  38.7125  38.7370  ...     38.651124             1.0  NaN
679   1609499940000  38.7259  38.7656  ...     38.647069             1.0  NaN
680   1609500000000  38.7547  38.7821  ...     38.645652             1.0  NaN
2371  1609601460000  38.0942  38.1100  ...     38.068003             1.0  1.0
3756  1609684560000  41.0045  41.0900  ...     40.872015             1.0  NaN

[5 rows x 18 columns]
               date     open     high  ...  bb_lowerband  open_indicator  won
2371  1609601460000  38.0942  38.1100  ...     38.068003             1.0  1.0
5425  1609784700000  39.6512  39.6800  ...     39.564646             1.0  1.0
6274  1609835640000  39.9462  39.9895  ...     39.868631             1.0  1.0
7100  1609885200000  40.7984  40.8064  ...     40.744386             1.0  1.0
7102  1609885320000  40.8025  40.8239  ...     40.741286             1.0  1.0

[5 rows x 18 columns]


In [ ]:
nbr_open_indicator = len(df_part[df_part["open_indicator"]==1])
nbr_won= len(df_part[df_part["won"]==1])

form = datetime.fromtimestamp(df_part[0]["date"])
print(f"from: {form}")
print(f"won: {1 + (nbr_won / nbr_open_indicator) * 100}%")


KeyError: ignored

In [ ]:
df_part['date_ms'] = pd.to_datetime(df_part['date'], unit='ms')
df_part.set_index('date_ms', inplace=True, drop=True)

In [ ]:
add_flag(df_part, add_plots, "close_indicator", "high", "red", "v", True)
add_flag(df_part, add_plots, "open_indicator", "low", "green", "^", False)
add_flag(df_part, add_plots, "won", "low", "purple", "*", False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# pkwargs = dict(type='candle', style='starsandstripes', addplot=add_plots)
# mpf.plot(df_part, figscale=2.5, volume=False,returnfig=True,**pkwargs)